In [ ]:
import socket
import threading
import cv2 as cv
import numpy as np

## Client Program that will connect to the server

In [ ]:

soc = socket.socket()
soc.bind(("", 1234))  # Here " " means localhost IP
server_ip = input(" Enter the Server IP adress -> ")
server_port = input("Enter the Server Port number -> ")

soc.connect((server_ip, int(server_port))) 
soc.send(b"connected")  # Here b is use to convert the data in the form of byte and send() sends it.
camIndex = 0 #Here 0 means the internal camera 
cam = cv.VideoCapture(camIndex) # Here camera will start and start capturing the video

# In the funtion below the Client will receive the data from server: Client <- Server
def receiver():
    framesLost = 0
    print("Connection Established")
    while True:
        framesLost += 1 # here it will count the frame lost
        data = soc.recv(100000000)  # receiving data with the size limit
        if(data == b'finished'):
            print("Connection END")
            cam.release() #here this will close the camera 
            soc.close() # here this will end the connection
        else:  
            photo =  np.frombuffer(data, dtype=np.uint8) # here data in the form of byte is converted into numpy array 
            if len(photo) == 640*480*3: # here setting the size if window
                cv.imshow('Hi I am Server', photo.reshape(480, 640, 3))
                if cv.waitKey(1) == 13: # here 13 is the code for Enter and as soon as we press enter it will stop the camera
                    soc.send(b'finished')
                    cam.release() #here camera is released or closed
                    cv.destroyAllWindows() #It will destroy the window as soon as we press the enter
                    break
            else:
                print("Lost {} frames".format(framesLost) ) #This will count the loss frame 

# In the function below the Client is connecting the server and sends the data: Client -> Server          
def sender():
    while True:  
        ret, photo = cam.read() #here it is reading the camera data
        photo = cv.resize(photo, (640, 480)) #here photo is resized
        print(photo.shape)
        if ret:
            soc.send(np.ndarray.tobytes(photo))
        else: print("No Frame Found")
    cam.release()

#Threading
threading.Thread(target=receiver).start()
threading.Thread(target=sender).start()
            

In [ ]:
cam.release()